In [ ]:
pip install git+https://github.com/facebookresearch/fastText.git

In [3]:
pip install fasttext-wheel


   ---------------------------------------- 0/2 [pybind11]
   ---------------------------------------- 0/2 [pybind11]
   ---------------------------------------- 0/2 [pybind11]
   ---------------------------------------- 0/2 [pybind11]
   -------------------- ------------------- 1/2 [fasttext-wheel]
   ---------------------------------------- 2/2 [fasttext-wheel]

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

df = pd.read_csv("datasets/cleanedDataSecondModel_merged.csv")

# Convert to fastText format
def save_fasttext_format(df, filepath):
    with open(filepath, 'w', encoding='utf-8') as f:
        for text, label in zip(df['quote'], df['general_label']):
            line = f"__label__{label} {text.strip()}\n"
            f.write(line)

save_fasttext_format(df.sample(frac=0.8, random_state=42), "datasets/FTtrain.txt")
save_fasttext_format(df.drop(df.sample(frac=0.8, random_state=42).index), "datasets/FTtest.txt")


In [1]:
import fasttext

model = fasttext.train_supervised(input="datasets/FTtrain.txt", epoch=4, lr=1, wordNgrams=4, verbose=2)
model.save_model("models/fasttext_model.bin")


In [2]:
model.test("datasets/FTtest.txt")


(200, 0.84, 0.84)

In [7]:
quote = "Success is not final, failure is not fatal"
label, prob = model.predict(quote)

print("Predicted Category:", label[0].replace("__label__", ""))
print("Confidence:", prob[0])


Predicted Category: Other
Confidence: 0.7860459089279175


In [8]:
from sklearn.metrics import classification_report, confusion_matrix
test_texts = []
true_labels = []

with open("datasets/FTtest.txt", "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split(" ", 1)
        if len(parts) == 2:
            label, text = parts
            test_texts.append(text)
            true_labels.append(label.replace("__label__", ""))


predicted_labels = []

for text in test_texts:
    label, _ = model.predict(text)
    predicted_labels.append(label[0].replace("__label__", ""))


print(classification_report(true_labels, predicted_labels))



               precision    recall  f1-score   support

     Altruism       0.00      0.00      0.00        13
     Learning       0.00      0.00      0.00        16
        Other       0.61      1.00      0.75        46
   Positivity       0.94      0.91      0.92        33
Relationships       1.00      1.00      1.00        38
         Self       1.00      1.00      1.00        54

     accuracy                           0.84       200
    macro avg       0.59      0.65      0.61       200
 weighted avg       0.75      0.84      0.79       200



c:\Users\VJ24-1\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\VJ24-1\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\VJ24-1\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,